# Preprocessing

> This module store preprocessing funtions. 


In [ ]:
#| default_exp preprocessing

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import logging
import os
import numpy as np
import pandas as pd

In [ ]:
from infepy.utils import read_k_file, read_csv_file

In [ ]:
#| export
logger = logging.getLogger(name='preprocessing')
logging.basicConfig(filename="preprocessing.log",filemode='w+' ,level=logging.DEBUG, force=True, format='[%(asctime)-15s] %(levelname)-8s %(filename)s %(funcName)s line %(lineno)d %(message)s')

In [ ]:
#| hide
import toml
from infepy.utils import _merge_path

In [ ]:
#| hide
def read_toml(  config_file= "Config.toml" # Path to the config file
):
    "Read setting file. The File containes the relative path to source and target."
    config= toml.load(config_file)
    return config
config  = read_toml()

In [ ]:
#| export
def read_nodes(path_to_file:str # path to file containing source template
               ) -> pd.DataFrame: # Numpy array of shape [n_nodes, x,y,z_displacement]
    "Read the nodes from the source template. The source template must be either a .key/.k file or .csv"    
    try:
        if path_to_file.endswith('.csv'):
            mesh_df = read_csv_file(path_to_file)
        elif path_to_file.endswith('.key') or path_to_file.endswith('.k'):
            mesh_df= read_k_file(path_to_file)
        return mesh_df
    except:
        logger.exception("Read_Nodes: No readable files")

In [ ]:
read_nodes(_merge_path(config['source']['path'],config['source']['filename_mesh']))

,Label - node id,x,y,z
0,591310,-181.550415,-145.430511,404.375488
1,3591330,-43.251625,-196.187363,194.874634
2,3591332,-37.076435,-219.493759,205.082413


In [ ]:
#| export
def read_landmarks(path_to_file:str # .csv or .key file containing Landmarks
                   )-> pd.DataFrame: # Dataframe of length [n_landmarks] divided in columns [ID - label, x,y,z]
    "Read the landmarks from .csv/.key/.k file."
    file_exist = False
    if path_to_file.endswith('.csv'):
        landmarks_df = read_csv_file(path_to_file)
        file_exist = True
    elif path_to_file.endswith('.key') or path_to_file.endswith('.k') :
        landmarks_df= read_k_file(path_to_file)
    
    assert not landmarks_df.empty
    return landmarks_df
# except:
    #     logger.exception("Read_Landamarks - No readable files")
       

In [ ]:
read_landmarks(_merge_path(config['source']['path'],config['source']['filename_landmarks']))

,Label - node id,x,y,z
0,1,100.001907,110.13913,-100.980621


In [ ]:
#| export
def _check_landmarks(source: pd.DataFrame, # Source dataframe
                    target: pd.DataFrame # Target dataframe
                    ):
    "This function compares the source and target Dataframe. It performs two test: if the have same amount of landmarks and if Label/IDs are in the same order.  "
    try:
        assert len(source) == len(target), "Not same amount of landmarks for source and target"
        
        bool = target.iloc[:,0].values == source.iloc[:,0].values
        assert bool.any() == True, "Order of landmarks is not the same for target and source"
        
        if bool.any() == False:
            return [i for i,x in enumerate(bool) if x==False] # => [1, 3]
            
        # TO DO: return values that are false. Bool gived false and extract index. Print. 
        # Sort them
        # if they dont match, return exception -->
    except:
        logger.exception("Invalid Landmarks. Need to be same size and order")       
    return

In [ ]:
_merge_path(config['target']['path'],config['target']['filename_landmarks'])

'data/target/landmarks_target.csv'

In [ ]:
source = read_landmarks(_merge_path(config['source']['path'],config['source']['filename_landmarks']))
target = read_landmarks(_merge_path(config['target']['path'],config['target']['filename_landmarks']))

In [ ]:
source

,Label - node id,x,y,z
0,1,100.001907,110.13913,-100.980621


In [ ]:
target

,Label - node id,x,y,z
0,2,100.001907,110.13913,-100.980621


In [ ]:
#| export
def write_output(morphed_mesh : np.ndarray, # Morphed mesh
                 morphed_file: np.ndarray, #  path to directory to save the file
                 mesh_file: np.ndarray, # path to the source mesh from read_setting()
                 ):
    "Write an output file for the morphed mesh in key file format."
    assert os.path.exists(morphed_file)
    
    if not os.path.exists(morphed_file): # write the file if doesn´t exist
        open(morphed_file, 'w').close()
     
    node_indicator = False
    idx_nodes = 0
    # Read the file and read line by line to find *node section.
    with open(mesh_file, 'r') as fp:
        file_content = fp.readlines()
    for idx_line, line in enumerate(file_content):
        if line.startswith('*NODE'):
            node_indicator = True
            continue
        if node_indicator and line.startswith('*'):
            break
        if node_indicator:
            line_list = list(line)
            line_list[8:24] = to_ls_dyna(morphed_mesh[idx_nodes, 0]) # x
            line_list[24:40] = to_ls_dyna(morphed_mesh[idx_nodes, 1]) # y
            line_list[40:56] = to_ls_dyna(morphed_mesh[idx_nodes, 2]) # z
            file_content[idx_line] = ''.join(line_list)
            idx_nodes += 1
    with open(morphed_file, 'w') as fp:
        for line in file_content:
            fp.write(line)
    return

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()